<a href="https://colab.research.google.com/github/Jarde01/ml-tools-and-examples/blob/master/troll_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install wandb pytorch-transformers

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 143kB 42.5MB/s 
     |████████████████████████████████| 235kB 44.1MB/s 
     |████████████████████████████████| 81kB 24.2MB/s 
     |████████████████████████████████| 460kB 39.6MB/s 
     |████████████████████████████████| 92kB 26.7MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 1.0MB 30.6MB/s 
     |████████████████████████████████| 655kB 37.7MB/s 
     |████████████████████████████████| 256kB 37.7MB/s 
     |████████████████████████████████| 71kB 23.1MB/s 
     |████████████████████████████████| 184kB 41.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1faef575bc2a1664
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b76495bc8f784c
  Stored in

In [0]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [0]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, Dense, Dropout, Activation, Embedding, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras import backend as K
from keras.preprocessing.text import *
from keras.preprocessing import *
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, LabelEncoder

In [3]:
!wandb login 2295caae2022fe6a431a755f74d1e9e8605848eb
import wandb
from wandb.keras import WandbCallback
wandb.init(project="troll-detection")

Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


W&B Run: https://app.wandb.ai/jeremy/troll-detection/runs/r02yninm

In [0]:
import torch
from pytorch_transformers import *

In [0]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [148]:
input_ids = torch.tensor([bert_tokenizer.encode("some words here ./ and some punctuation..")])
# last_hidden_states = bert_model(input_ids)[0]  # Models outputs are now tuples
last_hidden_states = bert_model(input_ids)[0]  # Models outputs are now tuples
vectors = last_hidden_states.detach().numpy()[0]
vectors.shape

(12, 768)

In [0]:
def pad_sentence(sentence:str, length:int=15, fill:str='word'):
  tokens = sentence.split()
  num_tokens = len(tokens)
  
  if num_tokens < length:
    for i in range(0, length-num_tokens):
      tokens.append(fill)
  else:
    tokens = tokens[:length]
#   assert(len(sentence)) == length
  
  return " ".join(tokens)

## Preprocess the training data

In [124]:
df = pd.read_json("/content/Dataset for Detection of Cyber-Trolls.json", lines=True)
df.head()

,annotation,content,extras
0,"{'notes': '', 'label': ['1']}",Get fucking real dude.,NaN
1,"{'notes': '', 'label': ['1']}",She is as dirty as they come and that crook ...,NaN
2,"{'notes': '', 'label': ['1']}",why did you fuck it up. I could do it all day...,NaN
3,"{'notes': '', 'label': ['1']}",Dude they dont finish enclosing the fucking s...,NaN
4,"{'notes': '', 'label': ['1']}",WTF are you talking about Men? No men thats n...,NaN


In [0]:
labels = df['annotation']
labels = labels.apply(lambda x: x['label'][0])

In [0]:
def bert_layer(x):
  input_ids = torch.tensor([bert_tokenizer.encode(x)])
  last_hidden_states = bert_model(input_ids)[0]
  final = last_hidden_states.detach().numpy()
  return final

inputs = df['content']
fixed_sent = inputs.apply(lambda x: pad_sentence(x, 30))
processed = fixed_sent.apply(lambda x: bert_layer(x))

In [0]:
import tensorflow as tf

def get_model():


  
#   inputs = Input(shape=(1,), dtype='string') 
# #   inputs = Lambda(lambda x: bert_layer(x), output_shape=(768,))
#   x = Lambda(lambda x: bert_layer(x), output_shape=(768,))(inputs)

# #   inputs = Lambda(lambda x: bert_layer(x), input_shape=(30,), dtype=tf.string)

#   x = Dense(64, activation='relu')(inputs)
#   outputs = Dense(2, activation='softmax')(x)  
  #   model = Model(inputs=inputs, outputs=outputs)

  
  model = Sequential()
  model.add(Dense(32, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
  return model            

In [0]:
model = get_model()

In [85]:
model.fit(np.array([["hey", "this is text two"]]))

ValueError: ignored

In [78]:
model.predict(["yoyoyo"])

AttributeError: ignored

In [0]:

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [0]:
import random

def run_trials_single():
  run_name = "" # what are you testing with this run
  notes = ""

  #hyperparams
  hyperparams = dict(
    embed_out =random.randint(8,256),
    dense_1_nodes = random.randint(8,256),
    dropout_1=random.uniform(0,0.95),
  )

  config= {
      "epochs": 50,
      "num_classes":num_classes,
      "num_words": num_words,
      "batch_size":batch_size,
      **hyperparams
  }

  model = get_model(num_classes, num_words, **hyperparams)

  wandb.init(project="16-class-ner", name=run_name, config=config, notes=notes)

  history = model.fit(x_train, y_train, batch_size=config['batch_size'], epochs=config['epochs'], 
            validation_split=0.2, callbacks=[WandbCallback()])

  scores = model.evaluate(x_test, y_test)
  wandb.log({'test_'+metric: scores[index] for index, metric in enumerate(model.metrics_names)})
  print("\n*** Finished! ***")

In [0]:
for x in range(0,100):
  run_trials_single()